In [1]:
import pandas as pd

dataset_name='selfdeploy24_incre'

protocol='http'

if(protocol=='http'):
    key_fields=['User-Agent', 'Accept', 'Url', 'Accept-Encoding', 'Version', 'Connection']
if(protocol=='tls'):
    key_fields=['handshake_random_bytes', 'handshake_time', 'handshake_ciphersuites', 'handshake_sessionid', 'extensions_hash_algs', 'extensions_supported_groups']

# Read the CSV file
file_path = 'result_by_service/proj_still_unknown_'+protocol+'_'+dataset_name+'_with_clusterid.csv'
df = pd.read_csv(file_path)
print(df)

result_df = pd.DataFrame()

#Align the unknown-class probe indices
grouped = df.groupby('y_true_organization_name')
for org_name, group in grouped:
    if (org_name =='censys' or org_name=='driftnet'):
        file_path = 'unknown_probe_index/data_'+protocol+'/'+dataset_name+'/'+org_name+'.csv'
        df2 = pd.read_csv(file_path)
        index_list=group['index_original']
        filtered_df = df2.loc[index_list]
        filtered_df['ClusterNo']=list(group['ClusterNo'])
    else:
        file_path = 'unknown_probe_index/data_'+protocol+'/'+dataset_name+'/unknown.csv'
        df2 = pd.read_csv(file_path)
        index_list=group['index_original']
        filtered_df = df2.loc[index_list]
        filtered_df['ClusterNo']=list(group['ClusterNo'])

    result_df = pd.concat([result_df,filtered_df], ignore_index=True)

print(result_df)

grouped_df = result_df.groupby('ClusterNo')

for cluster_index, group in grouped_df:
    print(f"Group {cluster_index}:")
    print('ips',len(set(group['src_ip'])),'packets',len(group))
    df = group
    #删除平台标签
    if('plat' in df.columns):
        del df['plat']
    del df['src_ip']
    
    df=df.fillna('N/A')
    # Create a list to store the unique values and frequents each field value appears
    types=[]
    counts = []
    
    for col in df.columns:
        c = df[col].value_counts()
        name=c.index
        counts.append(list(c))
        types.append(list(name))
        
    type_with_count=[]
    for i in range(len(counts)):
        if(len(counts[i])>0):
            mini_list=[]
            for j in range(len(counts[i])):
                mini_list.append(str(types[i][j])+':'+format(counts[i][j]/len(df)*100, '.1f')+'%')
            type_with_count.append(mini_list)
        else:
            type_with_count.append([])

    column_names = df.columns
    df_stat = pd.DataFrame(type_with_count).T
    df_stat.columns=column_names
    

    none_columns = df_stat.columns[df.isnull().all()]
    print(none_columns)
    for i in none_columns:
        del df_stat[i]

    #save to csv file
    if(protocol=='http' or protocol=='tls'):
        df_stat.head(3)[key_fields].to_csv('cluster_statistic/'+protocol+'_'+'stat'+'_'+str(int(cluster_index))+'.csv', index=False)
    else:
        df_stat.head(3).to_csv('cluster_statistic/'+protocol+'_'+'stat'+'_'+str(int(cluster_index))+'.csv', index=False)
        

                                                   proj  packet_pred  y_true  \
0     [-0.09518210589885712, -0.41751137375831604, -...           25      27   
1     [-0.09518210589885712, -0.41751137375831604, -...           25      27   
2     [-0.09518210589885712, -0.41751137375831604, -...           25      27   
3     [-0.09518210589885712, -0.41751137375831604, -...           25      27   
4     [-0.09518210589885712, -0.41751137375831604, -...           25      27   
...                                                 ...          ...     ...   
3490  [-1.0806291103363037, 0.441832572221756, -0.76...            4      26   
3491  [-0.5802410244941711, 0.1848473846912384, -1.1...           25      26   
3492  [-1.0806291103363037, 0.441832572221756, -0.76...            4      26   
3493  [-1.0806291103363037, 0.441832572221756, -0.76...            4      26   
3494  [0.5850507616996765, 0.09031827002763748, -1.1...            9      26   

                  ip proto  index_origi